In [70]:
import pandas as pd

# AERONEFS

In [71]:
dfa = pd.read_csv('data/aeronefs.csv')

display(dfa.shape)
display(dfa.head())

(2245, 5)

,ref_aero,type_model,debut_service,last_maint,en_maintenance
0,A330_0250,A330,2008-08-16,2023-01-08,False
1,B767_4309,B767,2011-08-01,2023-08-13,False
2,CRJ700_3824,CRJ700,2007-04-08,2023-11-29,True
3,E170_2901,E170,2012-11-20,2023-08-25,False
4,A321_1076,A321,2014-02-18,2022-12-04,False


# COMPOSANTS

In [72]:
dfc = pd.read_csv('data/composants.csv')

display(dfc.shape)
display(dfc.head())

(12257, 6)

,ref_compo,aero,desc,lifespan,taux_usure_actuel,cout_composant
0,C004,A320_2348,Éclairage d'urgence,14898,47.94,2910
1,C181,B777_2653,Système de gestion de carburant,13620,53.96,4112
2,C250,A350_5769,Dispositifs de levage (flaps),9456,63.59,3301
3,C654,A380_6627,Système de divertissement en vol,13584,73.25,7392
4,C142,B757_3104,Porte cargo,11439,2.77,4885


# DEGRADATION

In [73]:
start_date = '2023-11-29'
today = pd.Timestamp.today().strftime('%Y-%m-%d')

date_range = pd.date_range(start=start_date, end=today, freq='D')
# print(f"date_range:{date_range}")
df_each_date = []

for date in date_range:
    # print(date.strftime("%Y-%m-%d"))
    link = f'http://sc-e.fr/docs/degradations_{date.strftime("%Y-%m-%d")}.csv'
    
    try :
        df = pd.read_csv(link)
        df_each_date.append(df)
    except:
        continue

dfd = pd.concat(df_each_date)

display(dfd.shape)
display(dfd.head())

(2637, 6)

,ref_deg,linked_aero,compo_concerned,usure_cumulée,measure_day,need_replacement
0,D009477,B767_0318,C343,100.000,2023-11-29,True
1,D001145,B767_5911,C139,17.498,2023-11-29,False
2,D005001,B757_3104,C308,25.104,2023-11-29,False
3,D001380,A320_4610,C078,83.482,2023-11-29,True
4,D007247,E170_0751,C235,18.106,2023-11-29,False


# LOGS VOLS

In [74]:
dflv = pd.read_csv('data/logs_vols.csv')

display(dflv.shape)
display(dflv.head())

(32896, 6)

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V07297959,A350_0001,2021-09-30 19:47:33,5.2,"{'temp': '-23°C', 'pressure': '1006 hPa', 'vib...",1
1,V03905620,NaN,2020-12-29 00:08:35,3.4,"{'temp': '12°C', 'pressure': '1006 hPa', 'vibr...",1
2,V00138907,A330_2369,2023-06-02 01:37:26,5.4,"{'temp': '-2°C', 'pressure': '1048 hPa', 'vibr...",0
3,V00821768,B757_3261,2023-02-26 23:34:08,3.0,"{'temp': '-3°C', 'pressure': '994 hPa', 'vibra...",2
4,V09664649,E170_2901,2022-09-17 14:01:56,0.5,"{'temp': '50°C', 'pressure': '1097 hPa', 'vibr...",2


# ------------------ test sur A320_2348 ------------------

In [68]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A320_2348 = dfa[dfa['aero'] == "A320_2348"]
dfc_A320_2348 = dfc[dfc['aero'] == "A320_2348"]
dfd_A320_2348 = dfd[dfd['aero'] == "A320_2348"]
dflv_A320_2348 = dflv[dflv['aero'] == "A320_2348"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A320_2348 = pd.merge(dfc_A320_2348, dfd_A320_2348, on=common_columns)
df_A320_2348 = pd.merge(df_A320_2348, dfa_A320_2348, on='aero')
df_A320_2348 = pd.merge(df_A320_2348, dflv_A320_2348, on='aero')

# ------------------ test sur A320_0854 ------------------

In [69]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A320_0854 = dfa[dfa['aero'] == "A320_0854"]
dfc_A320_0854 = dfc[dfc['aero'] == "A320_0854"]
dfd_A320_0854 = dfd[dfd['aero'] == "A320_0854"]
dflv_A320_0854 = dflv[dflv['aero'] == "A320_0854"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A320_0854 = pd.merge(dfc_A320_0854, dfd_A320_0854, on=common_columns)
df_A320_0854 = pd.merge(df_A320_0854, dfa_A320_0854, on='aero')
df_A320_0854 = pd.merge(df_A320_0854, dflv_A320_0854, on='aero')